In [1]:
from utils.block import BlockRes
from tensorflow.keras import layers
from utils.common import compose
import tensorflow as tf

In [3]:
def CBN(
    filters, kernel_size=3,
    activation='relu', strides=1,
    padding="valid"):
    
    conv=layers.Conv2D(
        filters, kernel_size = kernel_size,
        strides = strides, padding = padding)
    bn=layers.BatchNormalization(epsilon = 1.001e-05)
    layers_list = [conv, bn]
    if activation:
        act=layers.Activation(activation = activation)
        layers_list.append(act)
    return compose(*layers_list)

In [13]:
def BlockRes(
    x, filters, kernel_sizes, strides = 1
    ):
    blocks = list()
    for idx, (filter, kernel_size) in enumerate(zip(filters, kernel_sizes)):
        stride = strides if idx == 0 else 1
        pad = "same" if idx != len(
            kernel_sizes) or len(kernel_sizes) == 2 else "valid"
        act = None if idx == len(kernel_sizes) else "relu"
        blocks.append(
            CBN(
                filter, kernel_size=kernel_size, 
                padding=pad, activation = act, strides = stride))            
    x1 = compose(*blocks)(x)
    if strides!=1 or (x.shape[-1]!=filters[-1]):
        x = CBN(
            filter, kernel_size=kernel_size, 
            strides = strides, activation = act, padding = pad)(x)
    x = layers.Add()([x1, x])
    x = layers.Activation(tf.keras.activations.relu)(x)
    return x

In [19]:
input_shape = (224,224,3)

inp = layers.Input(shape = input_shape)

In [20]:
x=compose(
    layers.ZeroPadding2D(padding=3),
    layers.Conv2D(64, 7, strides=2),
    layers.BatchNormalization(epsilon=1.001e-05),
    layers.Activation("relu"),
    layers.ZeroPadding2D( ),
    layers.MaxPooling2D(pool_size=3,strides=2)
)(inp)

In [21]:
kernel_sizes = [3, 3]
filters = [[64,64],[128,128],[256,256],[512,512]]
iters = [2, 2, 2, 2]
for i, (filter, iter) in enumerate(zip(filters, iters)):
    for j in range(iter):
        x = BlockRes(
            x, filter, kernel_sizes, 
            strides = 2 if (i != 0) &( j==0) else 1)

In [22]:
x = layers.GlobalAvgPool2D()(x)
out = layers.Dense(10, activation = "softmax")(x)
tf.keras.Model(inp, out).summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_6 (ZeroPadding2  (None, 230, 230, 3)  0          ['input_4[0][0]']                
 D)                                                                                               
                                                                                                  
 conv2d_26 (Conv2D)             (None, 112, 112, 64  9472        ['zero_padding2d_6[0][0]']       
                                )                                                             